In [2]:
import numpy as np
import matplotlib.pyplot as plt

from pyscf import ao2mo,gto, scf

from typing import Tuple


# qiskit
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP, NELDER_MEAD
from qiskit.algorithms import MinimumEigensolverResult

from qiskit import IBMQ, BasicAer, Aer
from qiskit.utils import QuantumInstance
from qiskit.quantum_info.operators import Operator
from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, CircuitStateFn
from qiskit.opflow.primitive_ops import PauliOp, PauliSumOp
from qiskit.quantum_info import Pauli, SparsePauliOp

# qiskit nature
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.mappers import ParityMapper, QubitConverter
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.problems import ElectronicBasis
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureMoleculeDriver, ElectronicStructureDriverType)

from qiskit_nature.settings import settings
settings.dict_aux_operators = True

#from qiskit import IBMQ
#from qiskit_nature.runtime import VQEClient
#from qiskit.providers.ibmq import least_busy
#from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session

# useful??
#from qiskit_nature.algorithms import (GroundStateEigensolver,
#                                      NumPyMinimumEigensolverFactory)
from qiskit.circuit.library import EfficientSU2
from qiskit.circuit.library import RealAmplitudes
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer import QasmSimulator
from qiskit import QuantumCircuit, transpile
from qiskit.opflow import TwoQubitReduction

def get_molecular_integrals(molecule):
    '''
    Calculates the 1-electron and 2-electron integrals.
    
    Args:   
        molecule : Mole 
            Ccorresponds to the molecule.
            
    Returns:
        h1_MO : ndarray
                Tensor corresponding to the 1-electron integral in the molecular basis.
        
        h2_MO : ndarray
                Tensor corresponding to the 2-electron integral in the molecular basis.

        n_particles: Tuple[int, int]
            Number of alpha and beta electrons.
        
        num_orb : int 
            Number of orbitals.

        nuclear_repulsion_energy : float
            Nuclear repulsion energy.
    '''
    
    mf = scf.RHF(molecule).run()

    # computing the 1e and 2e AO (atomic orbitals) integrals 
    h1 = molecule.intor_symmetric("int1e_nuc") + molecule.intor_symmetric("int1e_kin") 
    h2 = molecule.intor("int2e")
    # comment : add aosym (axis of symmetry) ???

    # transformations from AO to MO (molecular orbitals)
    h1_MO = np.einsum('pi,pq,qj->ij', mf.mo_coeff, h1, mf.mo_coeff)
    h2_MO = ao2mo.get_mo_eri(h2, mf.mo_coeff)

    #h2_MO = ao2mo.kernel(mol_h2, mf.mo_coeff, aosym="1") #alternative

    nuclear_repulsion_energy=molecule.energy_nuc()
    num_particles = molecule.nelec
    num_orb = h1.shape[0]

    return h1_MO, h2_MO, num_particles, num_orb, nuclear_repulsion_energy


def get_qubit_op(coordinates):
    '''
    Calculates the Hamiltonian as the sum of Pauli operators as a function
    of the coordinates of the atoms.
    
    Args:
        coordinates : ndarray
            Array containing the positions of the atoms constituing the molecule.
        
    
    Returns:
        qubit_op : PauliSumOp
            Sum of Pauli operators corresponding to the Hamiltonian.
        
        nelec : Tuple[int, int]
            Number of alpha and beta electrons.
        
        norb : int 
            Number of orbitals.
            
        problem : ElectronicStructureProblem
        
        converter : QubitConverter
    '''
    
    mol = gto.M(
                atom = [['H',(coordinates[0], coordinates[1], 0.0)], 
                        ['H',(coordinates[2], coordinates[3], 0.0)],
                        ['H',(coordinates[4], coordinates[5], 0.0)]],
                basis = 'sto3g',
                charge = 1,
                spin = 0,
                symmetry = True,
                verbose = 2 # control the print level of PySCF globally
    )

    h1_MO,h2_MO,nelec,norb,ecore = get_molecular_integrals(mol)

    # Define an ElectronicEnergy instance containing the 1e and 2e integrals
    electronic_energy = ElectronicEnergy.from_raw_integrals(
            h1_MO, ao2mo.restore(1, h2_MO, norb)
        )
    electronic_energy.nuclear_repulsion_energy = ecore

    # Define an ElectronicStructureProblem
    problem = ElectronicStructureProblem(electronic_energy)

    second_q_ops = problem.second_q_ops()     # get second quantized operators
    problem.num_spatial_orbitals = norb  # define number of orbitals
    problem.num_particles = nelec        # define number of particles 
    
    problem.basis = ElectronicBasis.MO 
    
    problem.molecule=MoleculeInfo(symbols=[mol.atom_symbol(i) for i in range(mol.nbas)], 
                                  coords=mol.atom_coords())
    FC_transformer=FreezeCoreTransformer(freeze_core=True)
    problem = FC_transformer.transform(problem)
    
    # TODO we need to give info about the molecule to use FreezeCoreTransformer 
    # question : how to do it in a smart way ? at the initialization ??
    
    hamiltonian = second_q_ops[0]  # Set electronic part of Hamiltonian
    hamiltonian = (hamiltonian + FermionicOp({"": ecore}, num_spin_orbitals=norb)) # add Nuclear Repulsion Energy
    #print("Hamiltonian :\n", hamiltonian) # print for checking purposes
    
    mapper = ParityMapper()  # Set Mapper
    
    # Do two qubit reduction
    converter = QubitConverter(mapper,two_qubit_reduction=True)
    qubit_op = converter.convert(hamiltonian, nelec)
    #print("q_op :\n", qubit_op) # print for checking purposes
    

    return qubit_op, nelec, norb, problem, converter

def calc_ground_state(qubit_op,nelec,norb,problem,converter) :
    '''
    Calculates the ground state and the ground energy of the Hamiltonian.
    
    Args:
        qubit_op : PauliSumOp
            Sum of Pauli operators corresponding to the Hamiltonian.
        
        nelec : Tuple[int, int]
            Number of alpha and beta electrons.
        
        norb : int 
            Number of orbitals.
            
        problem : ElectronicStructureProblem
        
        converter : QubitConverter
        
    
    Returns:
        ground_state : CircuitStateFn
            Ground state of the Hamiltonian.
            
        min_eng : float
            Ground energy of the Hamiltonian.
    '''

    backend = BasicAer.get_backend("statevector_simulator")
    quantum_instance = QuantumInstance(backend)
    optimizer = COBYLA(maxiter=200)

#     backend = Aer.get_backend('qasm_simulator') 
#     quantum_instance = QuantumInstance(backend,shots=1024)
#     optimizer = COBYLA(maxiter=200)

    init_state = HartreeFock(num_spatial_orbitals=norb, 
                                num_particles=nelec, 
                                qubit_converter=converter
                            )
    
    var_form = UCCSD(qubit_converter=converter,
                        num_particles=nelec,
                        num_spatial_orbitals=norb, 
                        initial_state=init_state
                        )

    vqe = VQE(var_form, optimizer, quantum_instance=quantum_instance) 
    vqe_result = vqe.compute_minimum_eigenvalue(qubit_op)
    min_eng = vqe_result.eigenvalue.real
    final_params = vqe_result.optimal_parameters 

    vqe_ground = vqe.ansatz.bind_parameters(final_params) 
#     vqe_ground_state = CircuitStateFn(vqe_ground)
    
    return vqe_ground, min_eng

In [4]:
Ha = 4.36e-18
angst = 1.0e-10
dR = 0.02 #removed the angst, since coordinates are given in angst


from qiskit.opflow import I, X, Y, Z
from qiskit.opflow.primitive_ops import PauliOp
from qiskit.quantum_info import Pauli
import numpy as np
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, CircuitStateFn
from qiskit import IBMQ, BasicAer, Aer

import time #to see which part of code takes the longest
import multiprocessing


#calculate centre of mass (currently for equal particle masses)
def calc_COM(coordinates):
    #print(coordinates)
    r_array = np.array_split(coordinates,3)
    COM = (r_array[0] + r_array[1] + r_array[2])/3
    
    return COM

#task to calculate expectation value
def task1(string, q_instance, psi_0):

    Obs = PauliOp(Pauli(string))
    measurable_expression = StateFn(Obs, is_measurement=True).compose(psi_0)
    expectation = PauliExpectation().convert(measurable_expression)  
    sampler = CircuitSampler(q_instance).convert(expectation) 
    E = sampler.eval().real
        
    return E

#task to calculate operator from coordinates
def task2(coordinates):
    
    #get qubit operators coresponding to H+ (hamiltonian for atomic distances R + dR)
    (qubit_op_plus, num_part_plus, num_orb_plus, problem_plus, converter_plus) = get_qubit_op(coordinates[0])
    #get qubit operators coresponding to H- (hamiltonian for atomic distances R - dR)
    (qubit_op_minus, num_part_minus, num_orb_minus, problem_minus, converter_minus) = get_qubit_op(coordinates[1])
       
    op =  (-(qubit_op_plus - qubit_op_minus))
                    
    op_list = op.primitive.to_list() #list containing Pauli operators in support and corresponding coeffs
    #print(op_list)    
    #construct dict from list
    op_dict = {}
    for (PauliString,coeff) in op_list:

        if PauliString in op_dict:
            op_dict[PauliString] += coeff
        else:
            op_dict[PauliString] = coeff
    
    return op, op_dict
 
def parallel_operators(coordinates):
    
    
    coordinate_list = []
    
    #COM = calc_COM(coordinates)
    #coords = np.array_split(coordinates,3)
    #print(coords)
    #print(COM)
    #v0 = coords[0] - COM
    #v1 = coords[1] - COM
    #v2 = coords[2] - COM
    
    #v0_normed = v0/np.sqrt(v0[0]**2+v0[1]**2)*dR
    #v1_normed = v1/np.sqrt(v1[0]**2+v1[1]**2)*dR
    #v2_normed = v2/np.sqrt(v2[0]**2+v2[1]**2)*dR
    
    #dR_list = v0_normed.tolist() + v1_normed.tolist() + v2_normed.tolist()
    
    dR_list = [dR,dR,dR,dR,dR,dR]
    #print(dR_list)
    for i in range(len(coordinates)):
        
        coordinate_list.append([coordinates[0:i]+[(coordinates[i]+dR_list[i])]+coordinates[i+1:],coordinates[0:i]+[(coordinates[i]-dR_list[i])]+coordinates[i+1:]])
    
    #print(coordinate_list)
    with multiprocessing.Pool() as pool:
         ops,op_coeffs = zip(*pool.map(task2, coordinate_list))
    
    #print(ops)
    return ops, op_coeffs


def parallel_expectations(exp_dict, op_set, q_instance, psi_0):

    triples = []
    for op in op_set:
        triple = (op,q_instance,psi_0)
        triples.append(triple)

    Es = 0
    with multiprocessing.Pool() as pool:
        Es = pool.starmap(task1, triples)

    for i in range(len(Es)):
        exp_dict[triples[i][0]]=Es[i]

    return 0

def sequential_expectations(exp_dict, op_set, q_instance, psi_0):

    for pauli_string in op_set:
        
        Obs = PauliOp(Pauli(pauli_string))
        measurable_expression = StateFn(Obs, is_measurement=True).compose(psi_0)
        expectation = PauliExpectation().convert(measurable_expression)  
        sampler = CircuitSampler(q_instance).convert(expectation) 
        E = sampler.eval().real
        exp_dict[pauli_string] = E

    return 0

def get_psi_0(coordinates, simulator='qasm',shots=1028):

    #get qubit operator corresponding to H(R)
    (qubit_op, num_part, num_orb, problem, converter) = get_qubit_op(coordinates)
    
    #get the ground state of H(R)
    psi_0,_ = calc_ground_state(qubit_op,num_part, num_orb, problem, converter)
    
    psi_0 = CircuitStateFn(psi_0)

    return psi_0

# def calc_forces(coordinates, psi=None, , shots=1024, simulator='qasm'):
    
    
#     psi_0 = psi
    
#     if (psi_0 == None):
#         psi_0 = get_psi_0(coordinates, simulator)
#     finish = time.time()
#     print("Took " + str(finish - start) + " seconds to find ground state")
#     if simulator == 'qasm':
#         backend = Aer.get_backend('qasm_simulator') 
#         q_instance = QuantumInstance(backend, shots=shots)
#     elif simulator == "exact":
#         backend = BasicAer.get_backend("statevector_simulator")
#         q_instance = QuantumInstance(backend)
#     else:
#         print('Error: simulator should be qasm or exact!')
        
#     forces = []
#     #loop over particles
#     force_ops,force_op_coeffs = parallel_operators(coordinates)
#     for i in range(len(coordinates)):
#         #get the expectation value Ã‚Â¨
        
#         measurable_expression = StateFn(Obs, is_measurement=True).compose(psi_0)
#         expectation = PauliExpectation().convert(measurable_expression)
 
#         sampler = CircuitSampler(q_instance).convert(expectation)

#         f = sampler.eval().real
#         #print(sampler_var.eval())
#         #variance = sampler_var.eval().real - f^2
#         forces.append(f*Ha*(1/(2*dR*angst)))
#         # in the dimension of nN to avoid small values.
    
#     return forces

def calc_force_and_cov(coordinates, simulator = 'qasm', PauliSet = None, mode = 'full', shots = 1024):
  
    
    if simulator == 'qasm':
            backend = Aer.get_backend('qasm_simulator')
            #backend.set_options(max_job_size=1, executor = euler_client)
            q_instance = QuantumInstance(backend, shots=shots)
    elif simulator == "exact":
            backend = BasicAer.get_backend("statevector_simulator")
            q_instance = QuantumInstance(backend)
    else:
        print('Error: simulator should be qasm or exact!')
        
    num_coord = len(coordinates)   
    
    start = time.time()

    psi_0 = get_psi_0(coordinates, simulator = simulator, shots=shots)
    
    end = time.time() 
    
    time_psi_0 = end - start
    #print("Took " + str(time_psi_0) + " s to calculate the ground state")
    forces = np.zeros(num_coord)
    coordinates = coordinates.tolist()
    
    start = time.time()
    #get force component operators and decomposition (without the 1/(2*dR) factor, for numerical reasons)
    #replace parallel_ with sequential_ to see effect of parallelization
    force_ops,force_op_coeffs = parallel_operators(coordinates)
    
    #for ops in force_ops:
        #print(len(ops))
        
    P1 = force_ops[0]
    
    if PauliSet == None:
        
        pauli_op_set = {'IIII'} #Could not initialize empty set for some reason
        # construct set of PauliOp products that we need to calculate

        for PauliString,coeff in force_op_coeffs[0].items():
            pauli_op_set.add(PauliString)

        #to get set of necessary PauliOp products:
        
            P_prod = (P1@P1)
            P_list = P_prod.primitive.to_list()

            for (PauliString,coeff) in P_list:
                pauli_op_set.add(PauliString)
                
    else: pauli_op_set = PauliSet
        
    end = time.time()
    
    time_pauli_ops = end - start
        

        # Now we calculate the expectation values of all the relevant PauliOps
    
    start = time.time()
    exp_dict = {} # dictionary with relevant expectation values
        
    #switch between parallel_ and sequential_ to see the effect
    
    parallel_expectations(exp_dict,pauli_op_set,q_instance,psi_0)

    
    for i in range(len(coordinates)):
        for pauli_op,coeff in force_op_coeffs[i].items():
            forces[i] += coeff.real*exp_dict[pauli_op]
            
    end = time.time()   
    time_calc_exp = end - start
    
    
    start = time.time()
    COV = np.zeros((num_coord,num_coord)) #force covariance matrix, without some multiplicative factors. 
        
    for i in range(num_coord):
        for j in range(i,num_coord):

            Pij = force_ops[i]@force_ops[j]
            Pij_list = Pij.primitive.to_list()

            #add product expectation 
            for (pauli_string,coeff) in Pij_list:
                COV[i,j] += coeff.real*exp_dict[pauli_string]
                    #subtract 
            COV[i,j] -= forces[i]*forces[j]

            #since COV is symmetric
            if i < j:
                COV[j,i] = COV[i,j]

#     for i in range(num_coord):
#         for j in range(i,num_coord,2):

#             Pij = (force_ops[i]@force_ops[j]).reduce()
#             Pij_list = Pij.primitive.to_list()

#             #add product expectation 
#             for (pauli_string,coeff) in Pij_list:
#                 COV[i,j] += coeff.real*exp_dict[pauli_string]
#                     #subtract 
#             COV[i,j] -= forces[i]*forces[j]

#             #since COV is symmetric
#             if i < j:
#                 COV[j,i] = COV[i,j]
                    
    end = time.time()
    
    time_cov = end - start

    print(f'Time to: Calculate Ground State - {round(time_psi_0,2)}s, Get Pauli Ops - {round(time_pauli_ops,2)}s, Calculate Pauli Expectation Values - {round(time_calc_exp,2)}s, Calculate Covariance - {round(time_cov,2)}s')
    #print(COV)
    
    return forces*Ha*(1/(2*dR*angst)),COV*(Ha*(1/(2*dR*angst)))**2

In [6]:
#from Quantech.verlet_integrator import distance
#from force import calc_force_and_cov
import numpy as np
from tqdm import tqdm #to show progress


#implementation of the explicit Euler integrator
femto = 1.0e-15
angst = 1.0e-10

def distance(coordinates1,coordinates2):
    return np.sqrt(np.power((coordinates1[0] - coordinates2[0]),2) + np.power((coordinates1[1] - coordinates2[1]),2))
    


# def main():
    
#     #mass of particle1 (H)
#     mass_0 = 1.67e-27
#     #mass of particle2 (H)
#     mass_1 = 1.67e-27
#     #mass of particle3 (H)
#     mass_2 = 1.67e-27
#     #time step of integrator
#     dt = 0.2*femto
#     #initial velocity of atoms
    
#     v_init_0 = np.array([-np.sqrt(3)/2*100,-1/2*100])
#     v_init_1 = np.array([np.sqrt(3)/2*100,-1/2*100])
#     v_init_2 = np.array([0,100])

#     v_init = np.concatenate((v_init_0,v_init_1,v_init_2),axis=None)

#     #distances between atoms
#     d = 0.8
#     #rotation angle of the initial triangle
#     b = 0/180*np.pi
    
#     #initial positions of atoms, arrangement = equilateral triangle
#     x1_init = np.array([-d/2*np.cos(b)+np.sin(b)*np.sqrt(3)/6*d,-d/2*np.sin(b)-np.cos(b)*np.sqrt(3)/6*d])+np.array([0,d*np.sqrt(3)/6])
#     x2_init = np.array([d/2*np.cos(b)+np.sin(b)*np.sqrt(3)/6*d,d/2*np.sin(b)-np.cos(b)*np.sqrt(3)/6*d])+np.array([0,d*np.sqrt(3)/6])
#     x3_init = np.array([-np.sin(b)*np.sqrt(3)/3*d,np.cos(b)*np.sqrt(3)/3*d])+np.array([0,d*np.sqrt(3)/6])
    
    
#     init_pos0 = x1_init
#     init_pos1 = x2_init
#     init_pos2 = x3_init
    
#     #print(distance(init_pos0,init_pos1))
#     #print(distance(init_pos0,init_pos2))
#     #print(distance(init_pos1,init_pos2))
    
#     init_pos = np.concatenate((init_pos0,init_pos1,init_pos2),axis=None)

#     #integrator timesteps
#     times = np.arange(0*femto, 60.0*femto, dt)
#     #coordinate array
#     coords = [init_pos]
#     force = []
#     distance01 = [d]
#     distance02 = [d]
#     distance12 = [d]
#     #temperature in ?? 
#     #k_B*T in Joule
#     k_b = 1.38065e-23
# #     T = 420
# #     k_T = k_b*T
# #delta factor
#     delta = 0.1*dt
#     #hold v_next
#     v_next = None
    
#     simulator = 'qasm'

#     for time in tqdm(times, desc='Progress: '):
        
#         r = coords[-1]
#         #print(r)
        
#         (forces,COV) = calc_force_and_cov(r, simulator = simulator, shots = 2048)
        
#         f1 = distance(np.array([forces[0],forces[1]]),np.array([0,0]))
        
#         f2 = distance(np.array([forces[2],forces[3]]),np.array([0,0]))
        
#         f3 = distance(np.array([forces[4],forces[5]]),np.array([0,0]))    
        
# #         print("f0,f1,f2 = " + str([f1,f2,f3]))
        
# #         print(forces)
# #         print(COV)
        
#         if time == 0:

#             v_next = v_init - delta/(2*k_T*np.power(mass_0*mass_1*mass_2,1/3))*np.matmul(COV,v_init)*dt + dt*forces/mass_0
            
#             r_next = (r*angst + dt*v_next)/angst

#             #r_split = np.array_split(r_next,3)

#             coords.append(r_next)

#         else:

#             v_next += - delta/(2*k_T*np.power(mass_0*mass_1*mass_2,1/3))*np.matmul(COV,v_next)*dt + dt*forces/mass_0
            
#             r_next = (r*angst + dt*v_next)/angst

#             #r_split = np.array_split(r_next,3)

#             coords.append(r_next)

#         dists = np.split(coords[-1],3)
#         d01 = distance(dists[0],dists[1])
#         distance01.append(d01)
#         d02 = distance(dists[0],dists[2])
#         distance02.append(d02)
#         d12 = distance(dists[1],dists[2])
#         distance12.append(d12)
#         print([d01, d02, d12])
        
#     return (distance01,distance02,distance12,times) 

In [ ]:
#mass of particle1 (H)
mass_0 = 1.67e-27
#mass of particle2 (H)
mass_1 = 1.67e-27
#mass of particle3 (H)
mass_2 = 1.67e-27

mass = np.array([mass_0, mass_0, mass_1, mass_1, mass_2, mass_2])

#time step of integrator
dt = 0.2*femto
#initial velocity of atoms

v_init_0 = np.array([-np.sqrt(3)/2*100,-1/2*100])
v_init_1 = np.array([np.sqrt(3)/2*100,-1/2*100])
v_init_2 = np.array([0,100])

v_init = np.concatenate((v_init_0,v_init_1,v_init_2),axis=None)
velocity = [v_init]

#distances between atoms
d = 0.8
#rotation angle of the initial triangle
b = 0/180*np.pi

#initial positions of atoms, arrangement = equilateral triangle
x1_init = np.array([-d/2*np.cos(b)+np.sin(b)*np.sqrt(3)/6*d,-d/2*np.sin(b)-np.cos(b)*np.sqrt(3)/6*d])+np.array([0,d*np.sqrt(3)/6])
x2_init = np.array([d/2*np.cos(b)+np.sin(b)*np.sqrt(3)/6*d,d/2*np.sin(b)-np.cos(b)*np.sqrt(3)/6*d])+np.array([0,d*np.sqrt(3)/6])
x3_init = np.array([-np.sin(b)*np.sqrt(3)/3*d,np.cos(b)*np.sqrt(3)/3*d])+np.array([0,d*np.sqrt(3)/6])


init_pos0 = x1_init
init_pos1 = x2_init
init_pos2 = x3_init


init_pos = np.concatenate((init_pos0,init_pos1,init_pos2),axis=None)

#integrator timesteps
times = np.arange(0*femto, 200.0*femto, dt)
#coordinate array
coords = [init_pos]
force = []
distance01 = [d]
distance02 = [d]
distance12 = [d]
#temperature in ?? 
#k_B*T in Joule
k_b = 1.38065e-23
# T = 420
# k_T = k_b*T
#delta factor
delta = 0.1*dt
#hold v_next
v_next = None
n_dof = 5
temperature = []

simulator = 'qasm'

for timet in tqdm(times, desc='Progress: '):

    r = coords[-1]
    #print(r)

    (forces,COV) = calc_force_and_cov(r, simulator = simulator, shots = 16384)

    f1 = distance(np.array([forces[0],forces[1]]),np.array([0,0]))

    f2 = distance(np.array([forces[2],forces[3]]),np.array([0,0]))

    f3 = distance(np.array([forces[4],forces[5]]),np.array([0,0]))    

#     print("f0,f1,f2 = " + str([f1,f2,f3]))

    #print(forces)
#     print(COV)

    if timet == 0:
        
        kin_eng = np.sum(np.array(velocity[-1])**2*mass/2)
        k_T = 2*kin_eng/len(velocity)/n_dof
        temperature.append(k_T/k_b)
        print(k_T/k_b)

        v_next = v_init - delta/(2*k_T*np.power(mass_0*mass_1*mass_2,1/3))*np.matmul(COV,v_init)*dt + dt*forces/mass_0
        r_next = (r*angst + dt*v_next)/angst

        #r_split = np.array_split(r_next,3)

        coords.append(r_next)
        velocity.append(v_next)

    else:
        
        kin_eng += np.sum(np.array(velocity[-1])**2*mass/2)
        k_T = 2*kin_eng/len(velocity)/n_dof
        temperature.append(k_T/k_b)
        print(k_T/k_b)

        v_next += - delta/(2*k_T*np.power(mass_0*mass_1*mass_2,1/3))*np.matmul(COV,v_next)*dt + dt*forces/mass_0
        r_next = (r*angst + dt*v_next)/angst

        #r_split = np.array_split(r_next,3)

        coords.append(r_next)
        velocity.append(v_next)

    dists = np.split(coords[-1],3)
    d01 = distance(dists[0],dists[1])
    distance01.append(d01)
    d02 = distance(dists[0],dists[2])
    distance02.append(d02)
    d12 = distance(dists[1],dists[2])
    distance12.append(d12)
    print([d01, d02, d12])

In [ ]:
from matplotlib import pyplot as plt

np_distance01 = np.array(distance01)
np_distance02 = np.array(distance02)
np_distance12 = np.array(distance12)

plt.plot(times,np_distance01[:-1],label = 'd01')
plt.plot(times,np_distance02[:-1],label = 'd02')
plt.plot(times,np_distance12[:-1],label = 'd12')

plt.title('Distances at time t')
plt.xlabel('Time')
plt.ylabel('Distance')
plt.legend()
plt.show()